In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import tensorflow as tf
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.preprocessing import image

import os
import numpy as np

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Set TensorFlow to use only the first GPU (GPU:0)
        tf.config.experimental.set_memory_growth(gpus[0], True)
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        print("Using GPU:", gpus[0])
    except RuntimeError as e:
        print(e)

Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [34]:
import tensorflow as tf
from tensorflow.keras import layers, Model

class UNet(Model):
    def __init__(self, input_size=(256, 256, 3), num_classes=1):
        super(UNet, self).__init__()

        # Encoder (Contracting Path) - Store before pooling for concatenation
        self.enc_conv1 = self.conv_block(64)
        self.enc_conv2 = self.conv_block(128)
        self.enc_conv3 = self.conv_block(256)
        self.enc_conv4 = self.conv_block(512)

        # Bottleneck
        self.bottleneck = self.conv_block(1024, pool=False)

        # Decoder (Expansive Path)
        self.upconv4 = self.upconv_block(512)
        self.upconv3 = self.upconv_block(256)
        self.upconv2 = self.upconv_block(128)
        self.upconv1 = self.upconv_block(64)

        # Output Layer (1x1 Convolution)
        self.output_layer = layers.Conv2D(num_classes, (1, 1), activation='sigmoid', padding='same')

    def conv_block(self, filters, pool=True):
      """Creates a convolutional block. Returns the feature map and pooled output."""
      conv = tf.keras.Sequential([
          layers.Conv2D(filters, (3, 3), padding='same'),
          layers.ReLU(),
          layers.Conv2D(filters, (3, 3), padding='same'),
          layers.ReLU(),
      ])
      pool_layer = layers.MaxPooling2D(pool_size=(2, 2)) if pool else None
      return conv, pool_layer  # Return both separately


    def upconv_block(self, filters):
        """Up-convolutional block."""
        return tf.keras.Sequential([
            layers.Conv2DTranspose(filters, (2, 2), strides=(2, 2), padding='same'),
            layers.ReLU(),
            layers.Conv2D(filters, (3, 3), padding='same'),
            layers.ReLU()
        ])

    def call(self, inputs):
      # Encoder (Contracting Path)
      enc1 = self.enc_conv1[0](inputs)  # Apply convolution block
      pool1 = self.enc_conv1[1](enc1)   # Apply pooling

      enc2 = self.enc_conv2[0](pool1)
      pool2 = self.enc_conv2[1](enc2)

      enc3 = self.enc_conv3[0](pool2)
      pool3 = self.enc_conv3[1](enc3)

      enc4 = self.enc_conv4[0](pool3)
      pool4 = self.enc_conv4[1](enc4)

      # Bottleneck
      bottleneck = self.bottleneck[0](pool4)

      # Decoder (Expansive Path)
      up4 = self.upconv4(bottleneck)
      up4 = layers.Concatenate()([up4, enc4])

      up3 = self.upconv3(up4)
      up3 = layers.Concatenate()([up3, enc3])

      up2 = self.upconv2(up3)
      up2 = layers.Concatenate()([up2, enc2])

      up1 = self.upconv1(up2)
      up1 = layers.Concatenate()([up1, enc1])

      # Output Layer
      output = self.output_layer(up1)

      return output

# Instantiate the model
unet = UNet(input_size=(256, 256, 3), num_classes=1)
unet.build(input_shape=(None, 256, 256, 3))
unet.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'u_net_12', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "u_net_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_108 (Sequential)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_48 (MaxPooling2D)      │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_109 (Sequential)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_49 (MaxPooling2D)      │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_110 (Sequential)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_50 (MaxPooling2D)      │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_111 (Sequential)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_51 (MaxPooling2D)      │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_112 (Sequential)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_113 (Sequential)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_114 (Sequential)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_115 (Sequential)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_116 (Sequential)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_194 (Conv2D)                  │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [35]:


# Instantiate the model

unet = UNet(input_size=(256, 256, 1), num_classes=1)

# Summary of the model architecture
unet.build((None, 256, 256, 1))  # Input shape
unet.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'u_net_13', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "u_net_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_117 (Sequential)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_52 (MaxPooling2D)      │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_118 (Sequential)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_53 (MaxPooling2D)      │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_119 (Sequential)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_54 (MaxPooling2D)      │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_120 (Sequential)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_55 (MaxPooling2D)      │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_121 (Sequential)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_122 (Sequential)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_123 (Sequential)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_124 (Sequential)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_125 (Sequential)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_209 (Conv2D)                  │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [36]:
def load_image(image_path):
    """Load an image, decode it, and resize to 256x256"""
    img = tf.io.read_file(image_path)               # Read image
    img = tf.image.decode_png(img, channels=3)       # Decode as PNG (assuming PNG images)
    img = tf.image.resize(img, (256, 256))           # Resize to target size
    img = img / 255.0                               # Normalize to range [0, 1]
    return img

def load_mask(mask_path):
    """Load a mask, decode it, and resize to 256x256"""
    mask = tf.io.read_file(mask_path)               # Read mask image
    mask = tf.image.decode_png(mask, channels=1)     # Decode as PNG (assuming masks are single-channel)
    mask = tf.image.resize(mask, (256, 256))         # Resize to target size
    mask = mask / 255.0                             # Normalize to range [0, 1]
    return mask

def load_data(image_folder, mask_folder):
    # Get image and mask file paths
    image_paths = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder)]
    mask_paths = [os.path.join(mask_folder, fname) for fname in os.listdir(mask_folder)]

    # Shuffle the paths (optional)
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))

    # Load images and masks
    dataset = dataset.map(lambda x, y: (load_image(x), load_mask(y)))

    # Shuffle and batch the dataset
    dataset = dataset.shuffle(buffer_size=1000)  # Shuffle with buffer_size (adjust as needed)
    dataset = dataset.batch(batch_size=16)  # Batch size for training (adjust as needed)

    return dataset


In [37]:
# Set the directory paths
image_folder = '/content/drive/MyDrive/CV project/normalized_images'
mask_folder = '/content/drive/MyDrive/CV project/normalized_masks'

# Load the dataset
train_dataset = load_data(image_folder, mask_folder)

# Check the dataset shapes (optional)
for images, masks in train_dataset.take(1):
    print("Image batch shape:", images.shape)
    print("Mask batch shape:", masks.shape)

# Compile the model
unet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
unet.fit(train_dataset, epochs=10)  # Adjust the number of epochs as needed

Image batch shape: (16, 256, 256, 3)
Mask batch shape: (16, 256, 256, 1)
Epoch 1/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 118s 997ms/step - accuracy: 0.7236 - loss: 0.5492
Epoch 2/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 36s 541ms/step - accuracy: 0.7968 - loss: 0.4743
Epoch 3/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 35s 543ms/step - accuracy: 0.8270 - loss: 0.4409
Epoch 4/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 35s 534ms/step - accuracy: 0.8243 - loss: 0.4447
Epoch 5/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 36s 538ms/step - accuracy: 0.8480 - loss: 0.3792
Epoch 6/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 35s 533ms/step - accuracy: 0.8447 - loss: 0.3871
Epoch 7/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 35s 533ms/step - accuracy: 0.8563 - loss: 0.3578
Epoch 8/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 35s 535ms/step - accuracy: 0.8626 - loss: 0.3521
Epoch 9/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 36s 533ms/step - accuracy: 0.8517 - loss: 0.3772
Epoch 10/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 40s 531ms/step - accuracy: 0.8683 - loss: 0.3439
